The **conditional generative adversarial network**(CGAN) is a type of GAN that involves the conditional generation of images by a generator model.
And this condition is based on the class label.The architecture of the conditional GAN is composed of generator and discriminator model.
As mentioned before that the generator is responsible for generating fake images that are very close to original image to deceive discriminator .The target of the discriminator is to distinguish between real and fake images.
The model is trained in adversarial manner . In other words , if the discriminator model shows improvement while dicriminating between real and fake images , this will affect negatively the cost of the generator.

The main targets from using the class labels are to improve the GAN(stable,faster training and better quality images) and targeted image generation.

There are some limitations applied on a GAN model as it may generate a random images from the domain and also there is a relationship between points in the latent space and the images generated by the generator.

Conditional GAN is  trained in  a way that both the generator and the discriminator models are conditioned by the class label which means that when the trained generator model is used as a standalone model to generate images in the domain,  and these images are of a given type, or class label, can be generated.

In [ ]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam # - Works
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
 

The discriminator model

An input is defined that takes an integer for the class label of the image to make the image conditional on the provided class label.After that this class lable is passed through embedding layer of size 50(50 classes maps into 50-elements)which will be learnt by the dicriminator model.And the output after that is passed to a FC layer with activation.after that the activations are reshaped into 28×28 activation map which is concatenated with the imput image


In [ ]:
 # The same as the previous methods but the only different thing is adding labels to the images by concatenate method

def MakeDiscriminator(inshape=(28,28,1), n_classes=10):
	inputlabel = Input(shape=(1,))
	li = Embedding(n_classes, 50)(inputlabel)
	
	nodes = inshape[0] * inshape[1]     #scaling image dimensions using linear activation
	li = Dense(nodes)(li)
 
	li = Reshape((inshape[0], inshape[1], 1))(li)

	inputimage = Input(shape=inshape)
	
	merge = Concatenate()([inputimage, li])        #the activations are reshaped into 28×28 activation map which is concatenated with the imput image
	

	featuremap = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)    #downsampling
	featuremap = LeakyReLU(alpha=0.2)(featuremap)                                    #LeakyRelu activation function
	
	featuremap = Conv2D(128, (3,3), strides=(2,2), padding='same')(featuremap)       #downsampling ,128 number of filters,(3,3)kernel,strides(2,2)that controls width and height
	
	featuremap = LeakyReLU(alpha=0.2)(featuremap)                                   
	
	featuremap = Flatten()(featuremap)                              # Flatten class: flattens the input but does not affect the batch size.
                                                                  #convert convolutional 2D layer into 1D of fully connected convolutional network

	
	featuremap = Dropout(0.4)(featuremap)                          #The Dropout layer randomly sets input units to 0 with a frequency of rate 
                                                                 #at each step during training time, which helps prevent overfitting
                                                                 #rate represents a Float between 0 and 1. Fraction of the input units to drop, in our
                                                                 #code it is 0.4
 
	output_layer = Dense(1, activation='sigmoid')(featuremap)
	

	model = Model([inputimage, inputlabel], output_layer)
	
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
 
	return model


The use of latent space conditional on the class label is to keep the generator model updated to take class label

the same as the discriminator this class lable is passed through embedding layer of size 50(50 classes maps into 50-elements)which will be learnt by the generator model.After that it will be passes to FC(fully connected) layer with linear activation.Then these activations are resized into  single 7×7 feature map which is added to 128 feature map to become 129 feature map that are unsampled.



Making Generator model

In [ ]:
# The same as the previous methods but the only different thing is adding labels to the images by concatenate method

def MakeGenerator(latent_dim, n_classes=10):
	
	input_label = Input(shape=(1,))
	
	li = Embedding(n_classes, 50)(input_label)
	
	nodes = 7 * 7
	li = Dense(nodes)(li)
	
	li = Reshape((7, 7, 1))(li)    #reshaping

	in_latent = Input(shape=(latent_dim,))
	
	nodes = 128 * 7 * 7
	image_gen = Dense(nodes)(in_latent)
	image_gen = LeakyReLU(alpha=0.2)(image_gen)
	image_gen = Reshape((7, 7, 128))(image_gen)
	
	merge = Concatenate()([image_gen, li])    #merging image with input label
	
	image_gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)    #upsampling to 14*14
	image_gen = LeakyReLU(alpha=0.2)(image_gen)
	
	image_gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(image_gen)    # upsample to 28x28
	image_gen = LeakyReLU(alpha=0.2)(image_gen)
	
	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(image_gen)          # 1 represents filters ,(7,7)represents strides
	
	model = Model([in_latent, input_label], out_layer)
	return model

Conditional GAN Making

The GAN model takes an input from a point in latent space and makes a prediction(real or fake)

The discriminator model will take an input (images generated from the generator and class label)


In [ ]:
 # we trained the conditional GAN by compiling the generator and discriminator model 
 
def MakeGAN(generator_model, discriminator_model):     #the GAN model(generator and discriminator)
	
	discriminator_model.trainable = False                 #weights are not trained in the discriminator model
	
	generator_noise, generator_label = generator_model.input    #input are taken from generator model(noise and labels)
	
	generator_output = generator_model.output
	
	GANoutput = discriminator_model([generator_output, generator_label])      #generated image and labels are the inputs given to the discriminator
	
	model = Model([generator_noise, generator_label], GANoutput)            #output classification
	
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model
 

Loading the dataset(fashion_minst)

In [ ]:

def LoadRealSamples():
	
	(X_train, Y_train), (_, _) = load_data()
	
	X = expand_dims(X_train, axis=-1)       #expand dimensions to 3dimension by adding channels
	
	X = X.astype('float32')
	
	X = (X - 127.5) / 127.5
	return [X,  Y_train]


 Chooses a batch of samples that must be updated to be used for real class labels from training dataset

In [ ]:

def GenerateRealSamples(dataset, num_samples):
	
	images, labels = dataset
	
	ix = randint(0, images.shape[0], num_samples)       #to select random instances
	
	X, labels = images[ix], labels[ix]
                                      
	y = ones((num_samples, 1))   #generating class for labels
	return [X, labels], y

This function should have all the updates to generate array of randomly choosen integer class labels to go along with the randomly selected points from the latent space


In [ ]:

def generate_latent_points(latent_dim, num_samples, n_classes=10):   #input=points from latent space
	
	Xinput = randn(latent_dim * num_samples)
	
	Zinput = 	Xinput.reshape(num_samples, latent_dim)     #reshaping
	
	labels = randint(0, n_classes, num_samples)        #generation of labels

	return [Zinput, labels]

This function is updated to use the randomly generated class labels as input to the generator model while generating fake images

In [ ]:

def FakeSamplesGeneration(generator, latent_dim, num_samples):      #the generator generates fake samples using the labels class
	
	Zinput, labels_input = generate_latent_points(latent_dim, num_samples)       #points generation in latent space
	
	images = generator.predict([Zinput, labels_input])            #output prediction
	
	y = zeros((num_samples, 1))
	return [images, labels_input], y

This function must be updated to use and retrieve the class labels in the calls to update both the generator model and the discriminator model 

In [ ]:

def train(generator_model, discriminator_model, GANmodel, dataset, latent_dim, n_epochs=2, n_batch=128):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	
	for i in range(n_epochs):          #  epochs enumeration
		
		for j in range(bat_per_epo):              #batch enumeration over the training set
			
			[X_real, labels_real], y_real = GenerateRealSamples(dataset, half_batch)       #select real samples randomly
			
			discriminator_loss1, _ = discriminator_model.train_on_batch([X_real, labels_real], y_real)       #discriminator model weights are updated
			
			[X_fake, labels], y_fake = FakeSamplesGeneration(generator_model, latent_dim, half_batch)    #fake samples generation
			
			discriminator_loss2, _ = discriminator_model.train_on_batch([X_fake, labels], y_fake)   # weights of discriminator model are updated 
			
			[Zinput, labels_input] = generate_latent_points(latent_dim, n_batch)     #inputs for the generator are the points from latent space
			
			y_gan = ones((n_batch, 1))                                   #inverted labels are created for fake samples
			
			generator_loss = GANmodel.train_on_batch([Zinput, labels_input], y_gan)    #generator is updated via discriminator errors
			
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, discriminator_loss1, discriminator_loss2,generator_loss))
	
	generator_model.save('cgan_generator.h5')

In [ ]:


latent_dim = 100   #size of latent space

discriminator_model = MakeDiscriminator(inshape=(28,28,1))

generator_model = MakeGenerator(latent_dim)

GANmodel = MakeGAN(generator_model, discriminator_model)

dataset = LoadRealSamples()

train(generator_model, discriminator_model, GANmodel, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, 1/468, d1=0.703, d2=0.696 g=0.691
>1, 2/468, d1=0.626, d2=0.700 g=0.687
>1, 3/468, d1=0.561, d2=0.707 g=0.680
>1, 4/468, d1=0.510, d2=0.720 g=0.668
>1, 5/468, d1=0.458, d2=0.736 g=0.656
>1, 6/468, d1=0.415, d2=0.760 g=0.637
>1, 7/468, d1=0.379, d2=0.789 g=0.627
>1, 8/468, d1=0.352, d2=0.802 g=0.624
>1, 9/468, d1=0.320, d2=0.799 g=0.657
>1, 10/468, d1=0.321, d2=0.731 g=0.727
>1, 11/468, d1=0.318, d2=0.644 g=0.832
>1, 12/468, d1=0.315, d2=0.557 g=0.938
>1, 13/468, d1=0.345, d2=0.522 g=0.980
>1, 14/468, d1=0.367, d2=0.529 g=0.928
>1, 15/468, d1=0.332, d2=0.584 g=0.832
>1, 16/468, d1=0.310, d2=0.634 g=0.765
>1, 17/468, d1=0.265, d2=0.681 g=0.712
>1, 18/468, d1=0.220, d2=0.745 g=0.657
>1, 19/468, d1=0.230, d2=0.854 g=0.583
>1, 20/468, d1=0.135, d2=0.970 g=0.522
>1, 21/468, d1=0.127, d2=1.042 g=0.503
>1, 22/468, d1=0.130, d2=1.035 g=0.541
>1, 23/468, d1=0.115, d2=0.899 g=0.655
>1, 24/468, d1=0.107, d2=0.696 g=0.857
>1, 25/468, d1=0.090, d2=0.507 g=1.108
>1, 26/468, d1=0.091, d2=0.395 g=1

In [ ]:
from math import floor
import numpy
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from keras.datasets import fashion_mnist
from skimage.transform import resize
from scipy.linalg import sqrtm

In [ ]:
# The below function is used in both IS and FID to resize the images to fit the inception model
# scale an array of images to a new size 
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

Inception Score Calculation

In [ ]:

# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):

	
	model = InceptionV3()     # loading inception v3 model
	
	scores = list()       # enumerate splits of images/predictions
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		
		ix_start, ix_end = i * n_part, (i+1) * n_part    # retrieve images
		subset = images[ix_start:ix_end]
		
		subset = subset.astype('float32')    # converting from uint8 to float32
		
		subset = scale_images(subset, (299,299,3)) # scaling images to the required size
		
		subset = preprocess_input(subset)    # pre-process images, scale to [-1,1]
		# predict p(y|x)
		p_yx = model.predict(subset)  # predict p(y|x)
		
		p_y = expand_dims(p_yx.mean(axis=0), 0)     	# calculate p(y)
		
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))  # calculate KL divergence using log probabilities
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1) # sum over classes
		
		avg_kl_d = mean(sum_kl_d)    # average over images
		
		is_score = exp(avg_kl_d)   #undo the log
		
		scores.append(is_score)   	# store
	# average across images
	is_avg, is_std = mean(scores), std(scores)     # average over images
	return is_avg, is_std
 
(images, _), (_, _) = fashion_mnist.load_data()
images = images[:100]

print('loaded', images.shape)
# calculation of the inception score
is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)

loaded (100, 28, 28)
96124928/96112376 [==============================] - 0s 0us/step
score 3.1019857 0.52531177


FID Calculation

In [ ]:

def calculate_fid(model, images1, images2):

	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
 
	# calculation of mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
 
	# calculation of sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
 
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
 
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real

	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

# preparation of the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

(images1, _), (images2, _) = fashion_mnist.load_data()

images1 = images1[:150]
images2 = images2[:150]

print('Loaded', images1.shape, images2.shape)

# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')

# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)

# preprocessing images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

# FID Calculation
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

Loaded (150, 28, 28) (150, 28, 28)
Scaled (150, 299, 299, 3) (150, 299, 299, 3)
FID: 81.764
